In [89]:
import numpy as np
from scipy.optimize import minimize, Bounds, LinearConstraint

In [91]:
# Asset returns
returnsArray = np.array([
    0.005,
    0.05,
    0.05,
    0.05,
    0.05,
])

# Asset volatility
volArray = np.array([
    .0,     # cash: 0% volatility,
    .12,
    .12,
    .12,
    .12,
])

# Asset correlations
asset1_asset2 = 1
asset1_asset3 = 1
asset1_asset4 = 1
asset2_asset3 = 1
asset2_asset4 = 1
asset3_asset4 = 1
corrArray = np.array([
    [1,             0,             0,             0,             0],
    [0,             1, asset1_asset2, asset1_asset3, asset1_asset4],
    [0, asset1_asset2,             1, asset2_asset3, asset2_asset4],
    [0, asset1_asset3, asset2_asset3,             1, asset3_asset4],
    [0, asset1_asset4, asset2_asset4, asset3_asset4,             1],
])

# Gamma - Risk coefficient. 1.0 = High risk (Kelly maximizing); 4.0+ = Low risk
gamma = 3.0


# Asset upper / lower bounds
cashBounds = (-1, 2)
asset2Bounds = (0, np.inf)
asset3Bounds = (0, np.inf)
asset4Bounds = (0, np.inf)
asset5Bounds = (0, np.inf)

assetBounds = [cashBounds] + [asset2Bounds] + [asset3Bounds] + [asset4Bounds] + [asset5Bounds]

def mean_and_std(asset_weights):
    r = np.dot(asset_weights, returnsArray)
    std = asset_weights * volArray
    std = std.reshape((1, len(std)))
    var = np.matmul(np.matmul(std, corrArray), std.transpose())
    return r, var[0][0]**.5


def utility(asset_weights):
    mean, std = mean_and_std(asset_weights)
    return mean - .5 * gamma * std**2


solution = scipy.optimize.minimize(
    lambda x: -utility(x),
    x0=np.full_like(returnsArray, 0.25),
    bounds=assetBounds,
    constraints=[{'type': 'eq', 'fun': lambda x: x.sum() - 1}, ],
    method='SLSQP'
)

print("asset weights: ", solution.x)

asset weights:  [-0.04166665  0.26041666  0.26041666  0.26041666  0.26041666]


In [88]:
i = range(5)

for n in i:
    print(round(solution.x[n], 3))

-0.042
0.26
0.26
0.26
0.26
